In [2]:
import gzip
import json

import numpy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
fl=gzip.open('goemotions.json.gz')
res=json.load(fl)
#test
print(res[2])

['Man I love reddit.', 'love', 'positive']


In [4]:
df=pd.DataFrame(res,columns=["post","emotion","sentiment"])
emotions_and_sentiments=df.drop(columns=['post'])

In [5]:
posts=df.drop(columns=['emotion','sentiment'])
numpy_posts=posts.to_numpy().flatten()
numpy_emotions_and_sentiments=emotions_and_sentiments.to_numpy()

In [6]:
numpy_emotions_and_sentiments[2][1]

'positive'

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =train_test_split(numpy_posts,numpy_emotions_and_sentiments,test_size=0.2,train_size=0.8,random_state=0)

In [8]:
y_train_emotion=y_train[:,0]
y_train_sentiment=y_train[:,1]
y_test_emotion=y_test[:,0]
y_test_sentiment=y_test[:,1]

In [10]:
import gensim.downloader as api
embedding=api.load("word2vec-google-news-300")

In [11]:
import nltk
from nltk import word_tokenize

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/safwanahmed/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

'comedycemetery'

In [12]:
def countTokens(X):
 token_word = []
 for words in X:
  token_word.extend(word_tokenize(words))
 return len(token_word)


In [13]:
X_train_tokens_count=countTokens(X_train)

In [14]:
X_test_tokens_count=countTokens(X_test)

In [15]:
def getEmbeddings(X,embed,size):
 posts_embedding=[]
 count=0
 for post in X:
  words=word_tokenize(post)
  word_embedding=[]
  for word in words:
    try:
     temp=embed[word]
     word_embedding.append(temp)
    except KeyError:
     count+=1
     pass
  accumumlate=[0]*size
  #change:do not hardcode some vectors may change size e.g. word2vec is 300
  accumumlate=np.array(accumumlate)
  for embedding_vec in word_embedding:
          accumumlate=embedding_vec+accumumlate
  if len(word_embedding)!=0:
     accumumlate=accumumlate/len(word_embedding)
  posts_embedding.append(numpy.copy(accumumlate))
 return posts_embedding,count






In [17]:
X_train_embeddings,X_train_misses=getEmbeddings(X_train,embedding,300)

In [18]:
X_test_embeddings,X_test_misses=getEmbeddings(X_test,embedding,300)

In [19]:
#77.45391136930904
#98.03390368826794
#84.55816358409068

hit_rate_train=100-((X_train_misses/X_train_tokens_count)*100)
print(hit_rate_train)

77.43935634073117


In [20]:
#77.43750557266085
#98.05362228796282
#84.46583863913162
hit_rate_test=100-((X_test_misses/X_test_tokens_count)*100)
print(hit_rate_test)

77.49562965116499


In [21]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
def write_matrix_and_report_to_file(y_test,y_pred,name,file,emotions):
 with open(file, 'a') as f:
  con=confusion_matrix(y_test,y_pred)
  f.write('\n')
  f.write(name)
  f.write('\n')
  if emotions:
   con_df=pd.DataFrame(con,index=['admiration','amusement','anger','annoyance','approval','caring','confusion','curiosity','desire','disappointment','disapproval','disgust','embarrassment','excitement','fear','gratitude','grief','joy','love','nervousness','neutral','optimism','pride','realization','relief','remorse','sadness','surprise'],columns=['admiration','amusement','anger','annoyance','approval','caring','confusion','curiosity','desire','disappointment','disapproval','disgust','embarrassment','excitement','fear','gratitude','grief','joy','love','nervousness','neutral','optimism','pride','realization','relief','remorse','sadness','surprise'])
  else:
    con_df=pd.DataFrame(con,index=['ambiguous','negative','neutral','positive'],columns=['ambiguous','negative','neutral','positive'])
  f.write('\n')
  rep=classification_report(y_test, y_pred,zero_division=0)
  f.write(rep)
  f.write('\n')
  f.write(con_df.to_string())
  f.write('\n')

# BASE MLP EMOTIONS word_2_vec-google-news


In [22]:
from sklearn.neural_network import MLPClassifier
neural_net_emotion=MLPClassifier().fit(X_train_embeddings,y_train_emotion)

/Users/safwanahmed/PycharmProjects/pythonProject/venv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [23]:
from sklearn.metrics import classification_report
y_pred_emotion=neural_net_emotion.predict(X_test_embeddings)

In [25]:
write_matrix_and_report_to_file(y_test_emotion,y_pred_emotion,'base_mlp_model_emotions_embeddings_word_2_vec-google-news-300','performance',True)

# BASE MLP SENTIMENTS word_2_vec-google-news

In [26]:
neural_net_sentiments=MLPClassifier().fit(X_train_embeddings,y_train_sentiment)

/Users/safwanahmed/PycharmProjects/pythonProject/venv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [27]:
y_pred_sentiments=neural_net_sentiments.predict(X_test_embeddings)

In [28]:
write_matrix_and_report_to_file(y_test_sentiment,y_pred_sentiments,'base_mlp_model_sentiments_embeddings_word_2_vec-google-news-300','performance',False)

# TOP MLP EMOTIONS word_2_vec-google-news

In [29]:
neural_net_emotion=MLPClassifier(activation='tanh',hidden_layer_sizes=(30,50),solver='adam').fit(X_train_embeddings,y_train_emotion)

/Users/safwanahmed/PycharmProjects/pythonProject/venv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [31]:
y_pred_emotion=neural_net_emotion.predict(X_test_embeddings)

In [32]:
write_matrix_and_report_to_file(y_test_emotion,y_pred_emotion,'top_mlp_model_emotions_embeddings_word_2_vec-google-news_activation=tanh,hidden_layer_sizes=(30,50),solver=Adam','performance',True)

# TOP MLP SENTIMENTS word_2_vec-google-news

In [30]:
neural_net_sentiments=MLPClassifier(activation='relu',hidden_layer_sizes=(30,50),solver='adam').fit(X_train_embeddings,y_train_sentiment)

/Users/safwanahmed/PycharmProjects/pythonProject/venv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [33]:
y_pred_sentiments=neural_net_sentiments.predict(X_test_embeddings)

In [34]:
write_matrix_and_report_to_file(y_test_sentiment,y_pred_sentiments,'top_mlp_model_sentiments_embeddings_word_2_vec-google-news_activation=relu,hidden_layer_sizes=(30,50),solver=Adam','performance',False)

# fasttext-wiki-news-subwords-300

In [35]:
embedding=api.load("fasttext-wiki-news-subwords-300")

In [36]:
X_train_embeddings,X_train_misses=getEmbeddings(X_train,embedding,300)

In [37]:
X_test_embeddings,X_test_misses=getEmbeddings(X_test,embedding,300)

In [38]:
hit_rate_train=100-((X_train_misses/X_train_tokens_count)*100)
print(hit_rate_train)

98.03846011829076


In [39]:
hit_rate_test=100-((X_test_misses/X_test_tokens_count)*100)
print(hit_rate_test)

98.03532525589667


In [40]:
neural_net_emotion=MLPClassifier(activation='tanh',hidden_layer_sizes=(30,50),solver='adam').fit(X_train_embeddings,y_train_emotion)
y_pred_emotion=neural_net_emotion.predict(X_test_embeddings)
neural_net_sentiments=MLPClassifier(activation='relu',hidden_layer_sizes=(30,50),solver='adam').fit(X_train_embeddings,y_train_sentiment)
y_pred_sentiments=neural_net_sentiments.predict(X_test_embeddings)

/Users/safwanahmed/PycharmProjects/pythonProject/venv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/safwanahmed/PycharmProjects/pythonProject/venv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [41]:
write_matrix_and_report_to_file(y_test_emotion,y_pred_emotion,'cv_mlp_model_sentiments_activation=TANH,hidden_layer_sizes=(30,50)_fasttext-wiki-news-subwords-300','performance',True)
write_matrix_and_report_to_file(y_test_sentiment,y_pred_sentiments,'cv_mlp_model_sentiments_activation=relu,hidden_layer_sizes=(30,50)_fasttext-wiki-news-subwords-300','performance',False)

# glove-twitter-100

In [42]:
embedding=api.load("glove-twitter-100")

In [43]:
X_train_embeddings,X_train_misses=getEmbeddings(X_train,embedding,100)

In [44]:
X_test_embeddings,X_test_misses=getEmbeddings(X_test,embedding,100)

In [45]:
hit_rate_train=100-((X_train_misses/X_train_tokens_count)*100)
print(hit_rate_train)

84.55000942046618


In [46]:
hit_rate_test=100-((X_test_misses/X_test_tokens_count)*100)
print(hit_rate_test)

84.49894847330405


In [47]:
neural_net_emotion=MLPClassifier(activation='tanh',hidden_layer_sizes=(30,50),solver='adam').fit(X_train_embeddings,y_train_emotion)
y_pred_emotion=neural_net_emotion.predict(X_test_embeddings)
neural_net_sentiments=MLPClassifier(activation='relu',hidden_layer_sizes=(30,50),solver='adam').fit(X_train_embeddings,y_train_sentiment)
y_pred_sentiments=neural_net_sentiments.predict(X_test_embeddings)

/Users/safwanahmed/PycharmProjects/pythonProject/venv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/safwanahmed/PycharmProjects/pythonProject/venv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [48]:
write_matrix_and_report_to_file(y_test_emotion,y_pred_emotion,'cv_mlp_model_sentiments_activation=tanh,hidden_layer_sizes=(30,50)_glove_twitter','performance',True)
write_matrix_and_report_to_file(y_test_sentiment,y_pred_sentiments,'cv_mlp_model_sentiments_activation=relu,hidden_layer_sizes=(30,50)_glove_twitter','performance',False)